## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-17-16-51-09 +0000,wapo,Supreme Court to consider case that could limi...,https://www.washingtonpost.com/politics/2025/1...
1,2025-11-17-16-49-43 +0000,nyt,Senator Accuses Kenya of Neglecting Children T...,https://www.nytimes.com/2025/11/17/world/afric...
2,2025-11-17-16-46-49 +0000,nypost,Boxer Goran Gogic’s NYC trial thrown into chao...,https://nypost.com/2025/11/17/us-news/boxer-go...
3,2025-11-17-16-45-00 +0000,wsj,Oil Giant TotalEnergies Is Betting Big on Elec...,https://www.wsj.com/business/deals/totalenergi...
4,2025-11-17-16-43-23 +0000,nyt,U.S. Border Patrol Arrested 81 People in Charl...,https://www.nytimes.com/2025/11/16/us/border-p...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2320/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
77,trump,34
187,epstein,12
461,new,12
83,files,11
156,up,9


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
160,2025-11-17-04-47-00 +0000,wsj,Trump threw in the towel on dissuading House R...,https://www.wsj.com/politics/policy/trump-play...,98
172,2025-11-17-02-44-24 +0000,nypost,Trump says House Republicans should vote to re...,https://nypost.com/2025/11/16/us-news/trump-sa...,84
119,2025-11-17-10-21-07 +0000,nyt,Trump Says House Republicans Should Vote to Re...,https://www.nytimes.com/2025/11/16/us/politics...,79
24,2025-11-17-15-34-23 +0000,bbc,Trump calls on House Republicans to vote to re...,https://www.bbc.com/news/articles/cvgv653v1vjo...,76
123,2025-11-17-10-00-16 +0000,nypost,House GOP circulates Epstein files memo counte...,https://nypost.com/2025/11/17/us-news/house-go...,74


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
160,98,2025-11-17-04-47-00 +0000,wsj,Trump threw in the towel on dissuading House R...,https://www.wsj.com/politics/policy/trump-play...
219,39,2025-11-16-21-02-08 +0000,latimes,Body of 7-year-old girl is found off Big Sur; ...,https://www.latimes.com/california/story/2025-...
203,39,2025-11-16-23-33-06 +0000,nypost,Vile antisemitic graffiti scrawled on Mexico’s...,https://nypost.com/2025/11/16/world-news/antis...
196,34,2025-11-17-00-00-21 +0000,bbc,Formula 1 tech used to rev-up train wi-fi spee...,https://www.bbc.com/news/articles/cqjwkdez71ro...
51,33,2025-11-17-14-27-00 +0000,wsj,Former Bangladesh Leader Sheikh Hasina Sentenc...,https://www.wsj.com/world/asia/former-banglade...
80,30,2025-11-17-12-49-26 +0000,cbc,Dozens of religious pilgrims from India feared...,https://www.cbc.ca/news/world/saudi-medina-acc...
8,27,2025-11-17-16-36-08 +0000,nypost,Judge orders DOJ to turn over grand jury trans...,https://nypost.com/2025/11/17/us-news/judge-or...
174,27,2025-11-17-02-30-26 +0000,nypost,Trump says Venezuela would ‘like to talk’ — as...,https://nypost.com/2025/11/16/world-news/trump...
248,27,2025-11-16-18-36-27 +0000,nyt,UN Climate Summit in Brazil Brings Tourism and...,https://www.nytimes.com/2025/11/16/world/ameri...
75,25,2025-11-17-13-08-00 +0000,wsj,Dassault Aviation Rises After Ukraine Agrees t...,https://www.wsj.com/world/europe/dassault-avia...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
